In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

df0 = pd.read_csv('../../assets/data/monsters.csv')
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1793 monsters in database


In [2]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go
from bs4 import BeautifulSoup

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=False,
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(family='sans-serif', size=14),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, 
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}
    
def save_fig_html(fig, format, name):
    file_name = f'./{name}.html'

    match format:
        case 'large':
            #fig.update_traces(marker=dict(size=6))
            fig.update_layout(font=dict(size=14))
        case 'small':
            #fig.update_traces(marker=dict(size=4))
            fig.update_layout(font=dict(size=10))

    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = f'plotly-div-{format}'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [5]:
# plot the number of monsters by CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='stack',
        xaxis=dict(
            title_text='challenge rating',
            range=[0, 31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='monsters',
            #range=[0, 35],
            #tick0=0, dtick=5,
            #minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

def x_decoder(x):
    match x:
        case 0:
            return 0
        case 0.125:
            return 0.125
        case 0.25:
            return 0.28125
        case 0.5:
            return 0.5625
        case 1:
            return 1.125
        case _:
            return x

def w_decoder(x):
    match x:
        case 0:
            return 0.125
        case 0.125:
            return 0.125
        case 0.25:
            return 0.1875
        case 0.5:
            return 0.375
        case 1:
            return 0.75
        case _:
            return 1.0


for g in ['generic','legendary','unique']:
    dfC = df0[df0['CR'].gt(0) & df0['Category'].isin([g])].groupby(['CR']).count()
    x = [x_decoder(cr) for cr in dfC.index]
    w = [w_decoder(cr) for cr in dfC.index]
    cr = dfC.index.to_list()
        
    fig.add_trace(go.Bar(
        x=x,
        y=dfC['Category'],
        width=w,
        name=g,
        customdata=dfC.index,
        hovertemplate = '<b>' + g + '</b><br>'
            + 'CR %{customdata}<br>'
            + 'Monsters %{y:,.0f}'
            + '<extra></extra>'
    ))
    

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-monster-distribution-large')
    save_fig_html(fig, format='small', name='fig-monster-distribution-small')